### 유저 소개글 중 유의미한 keyword를 가장 잘 추출하는 한국어 Pos-tagging 테스트

    테스트할 Konlpy 라이브러리
    - (1) Komoran
    - (2) Hannanum
    - (3) Kkma
    - (4) Mecab
    - (5) Okt
    - (6) Twitter

## `1. 데이터 로드`

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('user_Introduction.csv')
data

,user_id,Sex,Age,location,Introduction
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다. 안정적인 직장에 안정적인 ...
2,2,남성,30대,b,안녕하세요. 저는 부천 있는 중소기업 현장에서 근무하고 있습니다. 나이가 있는 관...
3,3,남성,40대,e,저는 개발자입니다. 저랑 남아있는 생을 행복하게 함께보낼 배우자를 찾습니다.
4,4,여성,20대,h,🌼🌻🌼 여자 입니다 🌻🌼🌻dd
...,...,...,...,...,...
3191,3191,남성,40대,h,저는 이것저것? 장사를 하고 이 어려운 시기 잘? 버티고 있습니다ㅜㅜ 그쪽도 그런가...
3192,3192,남성,50대,h,다정다감하며 늘 긍정적이며 후회없는 삶을 위해 노력하며 생활합니다. 서로에게 위로가...
3193,3193,남성,40대,o,"안녕하세요~ 천안에 살고 있는 이해심, 배려심 많은 순수남입니다. 앞만 보고 달려오..."
3194,3194,남성,40대,h,안녕하세요 평범한 남자 입니다 좋은 인연 되었으면 줗겠습니다 비흡연 이구요 친구같은...


In [3]:
df = data.copy()

## `2. 기본 전처리`

- (1) 비어 있는 결측치 항목 제거
- (2) 공백 '           ' 위와 같은 형태 제거
- (3) 특수문자, 이모지 제거

In [4]:
import re

In [5]:
def basic_preprocessing(df,col):
    df.drop(df[df[col].isnull()].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('Isnull?', df[col].isnull().sum())
    
    # 공백 제거
    df[col] = df[col].apply(lambda x: x.strip())
    
    # 특수문자, 단모음/단자음 제거
    df[col] = df[col].apply(lambda x: re.sub(r'[\n\r]', '',x))
    df[col] = df[col].apply(lambda x: re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',x))
    df[col] = df[col].apply(lambda x: re.sub('[a-zA-Z]','',x))
    df[col] = df[col].apply(lambda x: re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]','',x))    
    
    # utf-8 encode
    df[col] = df[col].apply(lambda x: x.encode('utf-8','ignore').decode('utf-8'))
    
    # 이모티콘 (이모지) 제거
    def del_emoji (text):
        only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE)
        return only_BMP_pattern.sub(r'', text)
    
    df[col] = df[col].apply(del_emoji)

    return df

In [6]:
pre_df = basic_preprocessing(df, 'Introduction')
print(pre_df.shape)
pre_df.head(3)

Isnull? 0
(3194, 5)


,user_id,Sex,Age,location,Introduction
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다 안정적인 직장에 안정적인 수...
2,2,남성,30대,b,안녕하세요 저는 부천 있는 중소기업 현장에서 근무하고 있습니다 나이가 있는 관계로...


In [7]:
display(pre_df[pre_df['Introduction']==''])
pre_df.drop(pre_df[pre_df['Introduction']==''].index,inplace=True)
print(pre_df.shape[0])

,user_id,Sex,Age,location,Introduction
130,130,여성,20대,h,
185,186,남성,20대,h,
250,251,남성,100대,f,
1584,1586,남성,40대,k,
2371,2373,여성,40대,k,


3189


## `3. 라이브러리 별 Pos-tagging`

#### `Komoran 사용`

    - pos 태깅에서 'NNG'만 사용
    - 길이 단어 길이 2 이상 4 이하만 

In [8]:
from konlpy.tag import Komoran

In [9]:
komoran = Komoran()

In [10]:
def komoran_tokenize(sent):
    try:
        words = komoran.pos(sent, join=True)
        words = [w for w in words if ('/NNG' in w and (6<=len(w)<=9))]
        words = [w.split('/')[0] for w in words]
        
    except :
        sent = 'error'
        words = sent
    return words

In [11]:
%%time

pre_df['Komoran'] = pre_df['Introduction'].apply(komoran_tokenize)
pre_df.head(3)

CPU times: user 10.9 s, sys: 74.3 ms, total: 11 s
Wall time: 3.86 s


,user_id,Sex,Age,location,Introduction,Komoran
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...,"[인연, 배려, 친구, 편안, 사람, 사람, 노력]"
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다 안정적인 직장에 안정적인 수...,"[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 직원,..."
2,2,남성,30대,b,안녕하세요 저는 부천 있는 중소기업 현장에서 근무하고 있습니다 나이가 있는 관계로...,"[중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 결혼]"


In [12]:
pre_df[pre_df['Komoran']=='error'].index

Int64Index([47, 122, 348, 971], dtype='int64')

In [13]:
pre_df.iloc[pre_df[pre_df['Komoran']=='error'].index]

,user_id,Sex,Age,location,Introduction,Komoran
47,47,남성,20대,i,,error
122,122,남성,30대,k,,error
351,352,남성,40대,i,안녕하세요 원료의약품을 취급하는 강소기업에서 현재 17년 재직하고 있구요 회사에서 ...,"[원료, 취급, 기업, 재직, 회사, 성실, 근무, 건강, 남자, 결혼, 전제, 만..."
974,976,여성,40대,i,혹시 모를 좋은 인연이있기를 기도해요 대인관계에있어서 대화배려바른인성과지혜로 노력하...,"[인연, 기도, 대화, 배려, 노력, 경제, 여유, 인생, 의절, 반려자, 운동, ..."


#### `Hannanum 사용`

    - pos 태깅에서 'N'만 사용
    - 길이 단어 길이 2 이상 4 이하만

In [14]:
from konlpy.tag import Hannanum

In [15]:
hannanum = Hannanum()

In [16]:
def hannanum_tokenize(sent):
    try:
        words = hannanum.nouns(sent)
        words = [w for w in words if 2<=len(w)<=4]
        
    except :
        sent = 'error'
        words = sent
    return words

In [17]:
# %%time

# pre_df['Hannanum'] = pre_df['Introduction'].apply(lambda x: hannanum.nouns(x))
# pre_df.head(3)

#### `Kkma 사용`

    - pos 태깅에서 'NNG'만 사용
    - 길이 단어 길이 2 이상 4 이하만

In [18]:
from konlpy.tag import Kkma

kkma = Kkma()

In [19]:
def kkma_tokenize(sent):
    try:
        words = kkma.pos(sent, join=True)
        words = [w for w in words if ('/NNG' in w and (6<=len(w)<=9))]
        words = [w.split('/')[0] for w in words]
        
    except :
        sent = 'error'
        words = sent
    return words

In [20]:
%%time

pre_df['kkma'] = pre_df['Introduction'].apply(kkma_tokenize)
pre_df.head(3)

CPU times: user 14min 42s, sys: 783 ms, total: 14min 42s
Wall time: 6min 39s


,user_id,Sex,Age,location,Introduction,Komoran,kkma
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...,"[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]"
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다 안정적인 직장에 안정적인 수...,"[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 직원,...","[중소기업, 동일, 업계, 회사, 임원, 안정적, 직장, 안정적, 수입, 회사, 발..."
2,2,남성,30대,b,안녕하세요 저는 부천 있는 중소기업 현장에서 근무하고 있습니다 나이가 있는 관계로...,"[중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 결혼]","[안녕, 부천, 중소기업, 현장, 근무, 나이, 관계, 장난, 넉살, 올해, 사람,..."


#### `Mecab 사용`

    - pos 태깅에서 'NNG'만 사용
    - 길이 단어 길이 2 이상 4 이하만

In [21]:
from konlpy.tag import Mecab 

In [22]:
mecab = Mecab()

In [23]:
def mecab_tokenize(sent):
    try:
        pos = mecab.pos(sent)
        words = [n for n,tag in pos if tag in ['NNG'] if 2<=len(n)<=4]
        
    except :
        sent = 'error'
        words = sent
    return words

In [24]:
%%time

df['mecab'] = df['Introduction'].apply(mecab_tokenize)
df.head(3)

CPU times: user 377 ms, sys: 1e+03 ns, total: 377 ms
Wall time: 408 ms


,user_id,Sex,Age,location,Introduction,Komoran,kkma,mecab
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...,"[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]"
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다 안정적인 직장에 안정적인 수...,"[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 직원,...","[중소기업, 동일, 업계, 회사, 임원, 안정적, 직장, 안정적, 수입, 회사, 발...","[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 발주,..."
2,2,남성,30대,b,안녕하세요 저는 부천 있는 중소기업 현장에서 근무하고 있습니다 나이가 있는 관계로...,"[중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 결혼]","[안녕, 부천, 중소기업, 현장, 근무, 나이, 관계, 장난, 넉살, 올해, 사람,...","[안녕, 중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 연애,..."


#### `Okt 사용`

    - pos 태깅에서 'NNG'만 사용
    - 길이 단어 길이 2 이상 4 이하만

In [25]:
from konlpy.tag import Okt

okt = Okt()

In [26]:
def okt_tokenize(sent):
    try:
        words = okt.pos(sent, join=True)
        words = [w for w in words if ('/Noun' in w and (7<=len(w)<=9))]
        words = [w.split('/')[0] for w in words]
        
    except :
        sent = 'error'
        words = sent
    return words

In [27]:
%%time

pre_df['okt'] = pre_df['Introduction'].apply(okt_tokenize)
pre_df.head(3)


CPU times: user 48.5 s, sys: 104 ms, total: 48.6 s
Wall time: 23.7 s


,user_id,Sex,Age,location,Introduction,Komoran,kkma,mecab,okt
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...,"[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 서로, 배려, 친구, 사람, 사람, 노력]"
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다 안정적인 직장에 안정적인 수...,"[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 직원,...","[중소기업, 동일, 업계, 회사, 임원, 안정적, 직장, 안정적, 수입, 회사, 발...","[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 발주,...","[중소기업, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 발주, 직원,..."
2,2,남성,30대,b,안녕하세요 저는 부천 있는 중소기업 현장에서 근무하고 있습니다 나이가 있는 관계로...,"[중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 결혼]","[안녕, 부천, 중소기업, 현장, 근무, 나이, 관계, 장난, 넉살, 올해, 사람,...","[안녕, 중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 연애,...","[부천, 중소기업, 현장, 근무, 나이, 관계, 장난, 보고, 올해, 사람, 알콩달..."


## 4. `pos tagging 결과 비교`

In [30]:
print(pre_df[pre_df['Komoran']=='error'].shape[0])
print(pre_df[pre_df['kkma']=='error'].shape[0])
print(pre_df[pre_df['mecab']=='error'].shape[0])
print(pre_df[pre_df['okt']=='error'].shape[0])

4
5
0
0


In [31]:
print(pre_df['Komoran'].apply(lambda x: x ==[]).sum())
print(pre_df['kkma'].apply(lambda x: x ==[]).sum())
print(pre_df['mecab'].apply(lambda x: x ==[]).sum())
print(pre_df['okt'].apply(lambda x: x ==[]).sum())

95
10
16
20


## 5. 결과

In [33]:
display(pre_df.head(5))

,user_id,Sex,Age,location,Introduction,Komoran,kkma,mecab,okt
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...,"[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 배려, 친구, 편안, 사람, 사람, 노력]","[인연, 서로, 배려, 친구, 사람, 사람, 노력]"
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다 안정적인 직장에 안정적인 수...,"[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 직원,...","[중소기업, 동일, 업계, 회사, 임원, 안정적, 직장, 안정적, 수입, 회사, 발...","[중소기업, 동일, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 발주,...","[중소기업, 업계, 회사, 임원, 안정, 직장, 안정, 수입, 회사, 발주, 직원,..."
2,2,남성,30대,b,안녕하세요 저는 부천 있는 중소기업 현장에서 근무하고 있습니다 나이가 있는 관계로...,"[중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 결혼]","[안녕, 부천, 중소기업, 현장, 근무, 나이, 관계, 장난, 넉살, 올해, 사람,...","[안녕, 중소기업, 현장, 근무, 나이, 관계, 장난, 올해, 사람, 연애, 연애,...","[부천, 중소기업, 현장, 근무, 나이, 관계, 장난, 보고, 올해, 사람, 알콩달..."
3,3,남성,40대,e,저는 개발자입니다 저랑 남아있는 생을 행복하게 함께보낼 배우자를 찾습니다,"[행복, 배우자]","[개발자, 행복, 배우자]","[개발자, 행복, 배우자]","[개발자, 배우자]"
4,4,여성,20대,h,여자 입니다,[여자],[여자],[여자],[여자]


- Komoran : error 4, 못잡는 것 95, 소요시간 10초
- kkma : error 5, 못잡는 것 10, 소요시간 15분
- mecab : error 0, 못잡는 것 16, 소요시간 0.37초
- Okt : error 0, 못잡는 것 20, 소요시간 23초

- 시간 mecab > Komoran > Okt > kkma
- output 만족도 Okt > mecab > Komoran==kkma